This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS, and direct air capture (DAC). Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. On the other hand, DAC leads to a true removal of CO2 from the atmosphere and not just a reduction of emissions. 

In [181]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [182]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [183]:
T = 10*24;# 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [184]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1)), PeriodLength = 240)

In [185]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel


In [186]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [187]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [188]:
solar_pv = Transformation{Electricity}(;
    id = :SolarPV,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    #### Note that this transformation does not have a stoichiometry balance because the sunshine is exogenous :-)
)
solar_pv.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = solar_pv,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
capacity_factor = solar_capacity_factor,
investment_cost = solar_inv_cost,
fixed_om_cost = solar_fom_cost,
constraints = [Macro.CapacityConstraint()]
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electricity}(:SolarPV, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:E => TEdge{Electricity}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.5176, 0.3503, 0.1105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240,

We can also assigne a battery storage to the node:

In [189]:
battery = Transformation{Storage}(;
id = :battery,
stoichiometry_balance_names = [:storage],
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
can_expand = true,
can_retire = false,
existing_capacity_storage = 0.0,
investment_cost_storage = battery_inv_cost_storage,
fixed_om_cost_storage = battery_fom_cost_storage,
min_duration = 1,
max_duration = 10,
constraints = [Macro.StorageCapacityConstraint(),Macro.StoichiometryBalanceConstraint()],
discharge_capacity_edge = :discharge
)

battery.TEdges[:discharge] = TEdge{Electricity}(;
id = :discharge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
investment_cost = battery_inv_cost,
fixed_om_cost = battery_fom_cost,
variable_om_cost = battery_vom_cost,
st_coeff = Dict(:storage=>1/battery_eff_down),
constraints = [Macro.CapacityConstraint()]
)

battery.TEdges[:charge] = TEdge{Electricity}(;
id = :charge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :input,
has_planning_variables = false,
st_coeff = Dict(:storage=>battery_eff_up)
)


TEdge{Electricity}(:charge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Storage}(:battery, 1:1:240, StepRange{Int64, Int64}[1:1:240], [:storage], Dict{Symbol, TEdge}(:charge => TEdge{Electricity}(#= circular reference @-3 =#), :discharge => TEdge{Electricity}(:discharge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], D

We also consider a hydrogen demand node:

In [190]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [191]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint()]
)

NGImport = Transformation{NaturalGas}(;
id = :NGImport,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
#### Note that this transformation does not have a stoichiometry balance because we are modeling exogenous inflow of NG
)

NGImport.TEdges[:ng_source] = TEdge{NaturalGas}(;
id = :ng_source,
node = ng_node,
transformation = NGImport,
direction = :output,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
has_planning_variables = false,
price = ng_fuel_price,
)

TEdge{NaturalGas}(:ng_source, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGas}(:NGImport, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:ng_source => TEdge{NaturalGas}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264,

We also define a CO2 node:

In [192]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
)

Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[])

We define a captured CO2 node to account for the inventory of captured CO2 (output from CO2 capture, input to SNG or potential storage):

In [193]:
co2_captured_node = Node{CO2Captured}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint()]
)

co2_captured_sink = Transformation{CO2Captured}(;
id = :co2_captured_sink,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
#### Note that this transformation does not have a stoichiometry balance because we are modeling just a sink
)

co2_captured_sink.TEdges[:co2_captured_flow] = TEdge{CO2Captured}(;
id = :co2_sink_flow,
node = co2_captured_node,
transformation = co2_captured_sink,
direction = :input,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
has_planning_variables = false,
)

TEdge{CO2Captured}(:co2_sink_flow, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{CO2Captured}(:co2_captured_sink, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:co2_captured_flow => TEdge{CO2Captured}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, Float64[], 0.0, false, 1.0, 1.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeC

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [194]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_heatrate,:emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [Macro.CapacityConstraint()]
)

ngcc.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_fuel_CO2)
)

ngcc.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[]), Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(#= circular reference @-3 =#), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict(:emis

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [195]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc_ccs.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [Macro.CapacityConstraint()]
)

ngcc_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),:captured_emissions=>ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate)
)

ngcc_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

ngcc_ccs.TEdges[:CO2_Captured]=TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missin

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [196]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

electrolyzer.TEdges[:Electrolyzer_E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(:Electrolyzer_E => TEdge{Electricity}(#= circular reference @-3 =#), :H2 => TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912],

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [197]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_heatrate,:emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

smr.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_fuel_CO2)
)

smr.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(#= circular reference @-3 =#), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict(

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [198]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr_ccs.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

smr_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),:captured_emissions=>smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate)
)

smr_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

smr_ccs.TEdges[:CO2_Captured] = TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(miss

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [199]:
electric_dac = Transformation{DACElectric}(;
                id = :Electric_DAC,
                time_interval = time_interval(CO2),
                stoichiometry_balance_names  = [:energy,:captured_co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electric_dac.TEdges[:CO2_Captured] = TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>electric_dac_power_input,:captured_co2=>1.0),
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

electric_dac.TEdges[:CO2_Input] = TEdge{CO2}(;
    id = :CO2_Input,
    node = co2_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:captured_co2=>1.0)
)

electric_dac.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>1.0,:captured_co2=>0.0)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{DACElectric}(:Electric_DAC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :captured_co2], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.Abstra

Now that we have implemented different technologies, we can create the JuMP model.

In [200]:
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs; electric_dac;NGImport;co2_captured_sink];

In [201]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [202]:
Macro.add_planning_variables!.(system,model);

In [203]:
Macro.add_operation_variables!.(system, model);

In [204]:
Macro.add_all_model_constraints!.(system, model);

┌ Warning: MaxNonServedDemandConstraint required for a node that does not have a non-served demand variable so MACRO will not create this constraint
└ @ Macro c:\Users\pecci\Code\MACRO-dev\src\node.jl:92


In [205]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [206]:
using Gurobi

In [207]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-18


In [208]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6492 rows, 6028 columns and 15962 nonzeros
Model fingerprint: 0x56425319
Coefficient statistics:
  Matrix range     [2e-02, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+04]
Presolve removed 3760 rows and 3769 columns
Presolve time: 0.01s
Presolved: 2732 rows, 2259 columns, 7234 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.400506e+05   0.000000e+00      0s
    1516    1.1086626e+09   0.000000e+00   0.000000e+00      0s

Solved in 1516 iterations and 0.02 seconds (0.01 work units)
Optimal objective  1.108662624e+09

User-callback calls 1577, time in user-callback 0.00 sec


The installed electrolyzer capacity is:

In [209]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

0.0

The installed SMR capacity is

In [210]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

1039.5627

The installed SMR CCS capacity is

In [211]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

0.0

The installed solar capacity in MW is:

In [212]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

0.0

The installed battery capacity in MW is:

In [213]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

99.0

The installed NGCC capacity in MW is:

In [214]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

12772.0

The installed NGCC CCS capacity in MW is:

In [215]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

0.0

The installed Electric DAC capacity in Tons CO2/h is:

In [216]:
Macro.value(Macro.capacity(electric_dac.TEdges[:CO2_Captured]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [217]:
base_emissions  = sum(Macro.value.(Macro.flow(ngcc.TEdges[:CO2]))) +  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2])))

959351.1366111384

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [218]:
atm_CO2_captured  = sum(Macro.value.(Macro.flow(electric_dac.TEdges[:CO2_Input])))

0.0

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [219]:
net_emissions = sum(Macro.value.(Macro.net_balance(co2_node)))

959351.136611139

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [220]:
base_CO2_captured  = sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2_Captured]))) + sum(Macro.value.(Macro.flow(electric_dac.TEdges[:CO2_Captured])))

0.0

However, if we add a net-zero CO2 cap accounting for both positive emissions from Natural gas power and H2, and negative emissions from DAC:

In [221]:
merge!(co2_node.rhs_policy,Dict(Macro.CO2CapConstraint=>0.0))
append!(co2_node.constraints,[Macro.CO2CapConstraint()])
Macro.add_model_constraint!(co2_node.constraints[1],co2_node,model)

1-dimensional DenseAxisArray{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, JuMP.ScalarShape},1,...} with index sets:
    Dimension 1, [1]
And data, a 1-element Vector{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, JuMP.ScalarShape}}:
 vFLOW_NGCC_CO2[1] + vFLOW_NGCC_CO2[2] + vFLOW_NGCC_CO2[3] + vFLOW_NGCC_CO2[4] + vFLOW_NGCC_CO2[5] + vFLOW_NGCC_CO2[6] + vFLOW_NGCC_CO2[7] + vFLOW_NGCC_CO2[8] + vFLOW_NGCC_CO2[9] + vFLOW_NGCC_CO2[10] + vFLOW_NGCC_CO2[11] + vFLOW_NGCC_CO2[12] + vFLOW_NGCC_CO2[13] + vFLOW_NGCC_CO2[14] + vFLOW_NGCC_CO2[15] + vFLOW_NGCC_CO2[16] + vFLOW_NGCC_CO2[17] + vFLOW_NGCC_CO2[18] + vFLOW_NGCC_CO2[19] + vFLOW_NGCC_CO2[20] + vFLOW_NGCC_CO2[21] + vFLOW_NGCC_CO2[22] + vFLOW_NGCC_CO2[23] + vFLOW_NGCC_CO2[24] + vFLOW_NGCC_CO2[25] + vFLOW_NGCC_CO2[26] + vFLOW_NGCC_CO2[27] + vF

In [222]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6493 rows, 6028 columns and 17162 nonzeros
Coefficient statistics:
  Matrix range     [2e-02, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+04]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1086626e+09   9.593511e+05   0.000000e+00      0s
    1755    1.1839163e+09   0.000000e+00   0.000000e+00      0s

Solved in 1755 iterations and 0.17 seconds (0.35 work units)
Optimal objective  1.183916322e+09

User-callback calls 1775, time in user-callback 0.00 sec


Now, the electrolyzer capacity is:

In [223]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

0.0

The SMR capacity is:

In [224]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

140.3193

The SMR CCS capacity is:

In [225]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

899.2434

Now the solar capacity is:

In [226]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

0.0

The installed battery capacity is:

In [227]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

99.00000000002925

The installed NG capacity is:

In [228]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

1.465341015276863

The installed NG CCS capacity is:

In [229]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

12770.534658984723

The installed Electric DAC capacity is:

In [230]:
Macro.value(Macro.capacity(electric_dac.TEdges[:CO2_Captured]))

568.6152189462838

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [231]:
sum(Macro.value.(Macro.flow(ngcc.TEdges[:CO2]))) +  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2])))

116260.31434665824

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [232]:
dac_CO2_captured  = Macro.value(sum(Macro.flow(electric_dac.TEdges[:CO2_Input])[t] for t in time_interval(CO2)))

116260.31434665878

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [233]:
net_emissions = sum(Macro.value.(Macro.net_balance(co2_node)))

-2.219735506514553e-10

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [234]:
Macro.sum(Macro.value.(Macro.flow(co2_captured_sink.TEdges[:co2_captured_flow])))

1.1601566114423336e6

We see that in a net-zero emission case with negative emissions from DAC, a combination of Natural Gas Power CCS and SMR H2 CCS with DAC is deployed.

Here is the full hourly time series of captured CO2:

In [235]:
Macro.value.(Macro.flow(co2_captured_sink.TEdges[:co2_captured_flow]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:240
And data, a 240-element Vector{Float64}:
 4298.2221779160245
 4100.840858944569
 3994.633107589647
 3964.471407797997
 4005.742881378621
 4110.524642421363
 4243.456956630351
 4356.520886437221
 4510.981290169821
 4664.850115488015
    ⋮
 5397.013624157439
 5237.521180134846
 5040.268382447719
 5010.344393804504
 5014.705028006306
 5065.236616726001
 5182.1823697603595
 4806.203471964039
 4488.057582260061

The outflow of electricity from ngcc_ccs is:

In [236]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:E]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:240
And data, a 240-element Vector{Float64}:
 10457.500632519728
  9914.534658984723
  9597.534658984723
  9437.534658984723
  9412.534658984725
  9535.534658984723
  9797.534658984723
 10034.534658984723
 10436.534658984723
 10830.534658984725
     ⋮
 12770.534658984723
 12770.534658984723
 12770.534658984723
 12770.534658984723
 12770.534658984723
 12770.534658984723
 12740.534658984725
 11787.534658984723
 10958.534658984723

The inflow of natural gas into ngcc_ccs is:

In [237]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:NG]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:240
And data, a 240-element Vector{Float64}:
 22771.396386243872
 21589.07817828128
 20898.80495639958
 20550.40206838358
 20495.964117131083
 20763.79883729338
 21334.308566419582
 21850.38034429328
 22725.74260043348
 23583.684712172882
     ⋮
 27808.069729366878
 27808.069729366878
 27808.069729366878
 27808.069729366878
 27808.069729366878
 27808.069729366878
 27742.744187863882
 25667.56948611858
 23862.40702258568

The outflow of CO2 (emission) from ngcc_ccs is:

In [238]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:240
And data, a 240-element Vector{Float64}:
 412.27211278619205
 390.86645029125685
 378.36917543125685
 372.0614026312569
 371.0758131312569
 375.9249134712569
 386.2538914312569
 395.5972798912569
 411.44555905125685
 426.9784495712569
   ⋮
 503.46019477125685
 503.46019477125685
 503.46019477125685
 503.46019477125685
 503.46019477125685
 503.46019477125685
 502.2774873712568
 464.70681563125686
 432.02466781125685

The outflow of CO2 (captured) from ngcc_ccs is:

In [239]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:240
And data, a 240-element Vector{Float64}:
 3710.449015075729
 3517.7980526213128
 3405.322578881313
 3348.552623681313
 3339.6823181813133
 3383.324221241313
 3476.285022881313
 3560.375519021313
 3703.010031461313
 3842.8060461413133
    ⋮
 4531.141752941313
 4531.141752941313
 4531.141752941313
 4531.141752941313
 4531.141752941313
 4531.141752941313
 4520.497386341313
 4182.361340681313
 3888.2220103013133